In [1]:
import pandas as pd
import os

Procedimiento para dividir el archivo de 5Gb en 20 partes para su carga incremental

In [ ]:
def dividir_json_grande(archivo_entrada, num_partes):
    # Contar el número total de líneas/objetos en el archivo JSON
    with open(archivo_entrada, 'r', encoding='utf-8') as archivo:
        total_lineas = sum(1 for _ in archivo)
    
    # Calcular el tamaño de cada parte
    tamaño_parte = total_lineas // num_partes
    if total_lineas % num_partes != 0:
        tamaño_parte += 1

    # Leer y dividir el archivo en partes
    with open(archivo_entrada, 'r', encoding='utf-8') as archivo:
        for i in range(num_partes):
            archivo_salida = f'parte_{i + 1}.json'
            with open(archivo_salida, 'w', encoding='utf-8') as salida:
                salida.write('[')  # Comenzar un array JSON
                for j in range(tamaño_parte):
                    linea = archivo.readline()
                    if not linea:
                        break
                    salida.write(linea.rstrip())
                    if j < tamaño_parte - 1 and not archivo.tell() == total_lineas:
                        salida.write(',')
                salida.write(']')  # Terminar el array JSON
            # Si el archivo original ha terminado, salir del bucle
            if not linea:
                break

# Usar la función para dividir el archivo 'archivo.json' en 10 partes
dividir_json_grande('D:/PROYECTO FINAL/Yelp/review.json', 20)

In [2]:
df_reviews_yelp = pd.read_json('D:/PROYECTO FINAL/Yelp/review/parte_16.json')
df_reviews_yelp.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,g2bgAoRUeKWkDQHmpWpjMA,HIZvBs4nmsafwdhuWWtrdQ,uYBHf5LLq3f7YbBjncQdCg,4,1,0,1,"Friendly people, great selection of pastries, ...",2019-01-09 15:12:59
1,j4iWM8Ybfu6wFl54CdEIbw,z8UXeMDV-2cJOddzAcHDFg,DOoJrrsqGzZN2FqzvllKnQ,5,0,0,0,"Fantastic food! Wish they were in Fairfax, Va...",2019-05-12 00:25:30
2,MAkeW6gz1KwqasOqdqruCQ,OJHRfPll8s9bbhITS5RWIQ,vznHFDZ8-oZbJRU6Bslsdw,4,1,0,0,If you're a Dolphin Tale fan then this is a mu...,2018-07-16 14:55:27


In [3]:
df_reviews_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349514 entries, 0 to 349513
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    349514 non-null  object        
 1   user_id      349514 non-null  object        
 2   business_id  349514 non-null  object        
 3   stars        349514 non-null  int64         
 4   useful       349514 non-null  int64         
 5   funny        349514 non-null  int64         
 6   cool         349514 non-null  int64         
 7   text         349514 non-null  object        
 8   date         349514 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 24.0+ MB


In [4]:
df_estados_USA= pd.read_csv("D:/PROYECTO FINAL/Google Maps/estados_usa.csv",delimiter = ';', encoding = "utf-8")
df_estados_USA= df_estados_USA.rename(columns={'nombre_largo': 'estado', 'nombre_corto':'state'}) # cambiar nombre de la columna
df_estados_USA= df_estados_USA.drop(['codigos'], axis=1) 
df_estados_USA['estado']= df_estados_USA['estado'].convert_dtypes(convert_string=True)
df_estados_USA['estado']= df_estados_USA['estado'].str.strip()  # quita los espacios vacios

In [5]:
df_business = pd.read_pickle("D:/PROYECTO FINAL/Yelp/business.pkl")
# Se cargan los business para filtrar los reviews de la categoria restaurantes solamente
df_business = df_business.loc[:, ~df_business.columns.duplicated()]
df_business.drop_duplicates('business_id',inplace=True)
df_business['categories'] = df_business['categories'].str.split(',')
df_business= df_business.dropna(subset=['categories']) # Elimina datos nulos de la columna
df_business['Es_Restaurant'] = df_business['categories'].apply(lambda x: 'Restaurants' in x)
df_business= df_business[df_business['Es_Restaurant']]
df_business= df_business.dropna(subset=['state']) # Elimina datos nulos de la columna
df_business['state']= df_business['state'].str.strip()  # quita los espacios vacios

In [6]:
df_business = df_business.merge(df_estados_USA, on='state', how='left')

In [7]:
df_business.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,estado
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",Restaurants,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",California
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...",Restaurants,"{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",Arizona
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,TN,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...",Restaurants,None,Tennessee


In [8]:
lista_estados= ['Florida', 'Pennsylvania', 'Tennessee', 'California', 'Texas', 'New York']
df_business= df_business[df_business['estado'].isin(lista_estados)]

In [29]:
df_business.shape

(26893, 15)

In [9]:
df_business= df_business[['business_id', 'name']]

In [31]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26893 entries, 0 to 52266
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  26893 non-null  object
 1   name         26893 non-null  object
dtypes: object(2)
memory usage: 630.3+ KB


In [10]:
df_reviews_yelp = df_reviews_yelp.merge(df_business, on='business_id', how='inner')

In [11]:
df_reviews_yelp = df_reviews_yelp[(df_reviews_yelp['date'].dt.year >= 2010) & (df_reviews_yelp['date'].dt.year <= 2021)]

In [12]:
df_reviews_yelp.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,name
0,j4iWM8Ybfu6wFl54CdEIbw,z8UXeMDV-2cJOddzAcHDFg,DOoJrrsqGzZN2FqzvllKnQ,5,0,0,0,"Fantastic food! Wish they were in Fairfax, Va...",2019-05-12 00:25:30,Taco Mamacita
1,ax8iHpMuYf57Tja498oETQ,cs7C4VJ5ghS3CRrNBSgeMg,DOoJrrsqGzZN2FqzvllKnQ,4,0,0,0,I've eaten here 5 times or so now and it never...,2019-07-28 14:00:31,Taco Mamacita
2,YApRFp3-1OFLUcrCCIc8hg,T62QEApuG0AvdeZhVLtL2w,DOoJrrsqGzZN2FqzvllKnQ,5,0,0,0,Nom Nom Nom!\n\nI love this place . Great marg...,2015-06-20 17:29:14,Taco Mamacita


In [13]:
df_reviews_yelp= df_reviews_yelp.drop(['review_id','name'], axis=1)       #Elimina la columnas
df_reviews_yelp['platform']= 2
df_reviews_yelp= df_reviews_yelp[['business_id','user_id', 'date', 'stars','useful','funny','cool', 'text', 'platform']]

In [14]:
df_reviews_yelp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110303 entries, 0 to 112639
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   business_id  110303 non-null  object        
 1   user_id      110303 non-null  object        
 2   date         110303 non-null  datetime64[ns]
 3   stars        110303 non-null  int64         
 4   useful       110303 non-null  int64         
 5   funny        110303 non-null  int64         
 6   cool         110303 non-null  int64         
 7   text         110303 non-null  object        
 8   platform     110303 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 8.4+ MB


In [15]:
df_reviews_yelp.head(3)

,business_id,user_id,date,stars,useful,funny,cool,text,platform
0,DOoJrrsqGzZN2FqzvllKnQ,z8UXeMDV-2cJOddzAcHDFg,2019-05-12 00:25:30,5,0,0,0,"Fantastic food! Wish they were in Fairfax, Va...",2
1,DOoJrrsqGzZN2FqzvllKnQ,cs7C4VJ5ghS3CRrNBSgeMg,2019-07-28 14:00:31,4,0,0,0,I've eaten here 5 times or so now and it never...,2
2,DOoJrrsqGzZN2FqzvllKnQ,T62QEApuG0AvdeZhVLtL2w,2015-06-20 17:29:14,5,0,0,0,Nom Nom Nom!\n\nI love this place . Great marg...,2
